<a href="https://colab.research.google.com/github/pradeep2c1/Machine-Translation-model/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

import re

In [1]:
!nvidia-smi

Sun Jan  8 07:56:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

# Data pre-processing

In [4]:
lines = pd.read_csv("Hindi_English_Truncated_Corpus.csv", encoding='utf-8')

In [5]:
lines['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [6]:
lines = lines[lines['source'] == 'ted']
lines.head(10)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,to turn on the lights or to bring him a glass ...,"लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [7]:
lines.drop(columns = ['source'], inplace = True)
lines.shape

(39881, 2)

In [8]:
lines.reset_index(inplace = True)
lines.drop(columns = ['index'], inplace = True)

In [9]:
pd.isnull(lines).sum()

english_sentence    0
hindi_sentence      0
dtype: int64

In [10]:
lines.drop_duplicates(inplace=True)
lines.shape

(38803, 2)

In [11]:
lines.head()

,english_sentence,hindi_sentence
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
3,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
4,So there is some sort of justice,तो वहाँ न्याय है


In [12]:
# Make all english letters lowercase
lines['english_sentence'] = lines['english_sentence'].apply(lambda x: x.lower())

# Remove the quotes
lines['english_sentence'] = lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

# Remove special characters
sp_char = set(string.punctuation)
lines['english_sentence'] = lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in sp_char))
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in sp_char))

In [13]:
# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [14]:
# Add START and END tokens to the beginning and end of the target sequence
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: 'START_ ' + x + ' _END')

In [15]:
lines.head()

,english_sentence,hindi_sentence
0,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...
1,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...
2,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...
3,and who are we to say even that they are wrong,START_ और हम होते कौन हैं यह कहने भी वाले कि व...
4,so there is some sort of justice,START_ तो वहाँ न्याय है _END


In [16]:
lines = lines.sample(frac = 1).reset_index(drop = True)
lines.head()

,english_sentence,hindi_sentence
0,they picked up these photographs from the fore...,START_ उन्होंने जमीन से इन तस्वीरों को उठाया _END
1,and after a few months went by,START_ और फिर थोड़े महीनो के बाद _END
2,really small photos just that size,START_ वाकई छोटे फोटो बस इतने बड़े _END
3,so having experimented in all sorts of remote ...,START_ तो हर तरह की दूरदराज़ जगहों पर प्रयोग क...
4,that very early on into her treatment,START_ कि इलाज के बहुत पहले _END


# Tokenizing the data

In [17]:
en = set()
for sentence in lines['english_sentence']:
    for word in sentence.split():
        if word not in en:
            en.add(word)

hi = set()
for sentence in lines['hindi_sentence']:
    for word in sentence.split():
        if word not in hi:
            hi.add(word)

In [18]:
print('Unique words in English are : ', len(en))
print('Unique words in Hindi are : ', len(hi))

Unique words in English are :  17759
Unique words in Hindi are :  22782


In [19]:
lines['length_en'] = lines['english_sentence'].apply(lambda x: len(x.split()))
lines['length_hi'] = lines['hindi_sentence'].apply(lambda x: len(x.split()))

lines.head()

,english_sentence,hindi_sentence,length_en,length_hi
0,they picked up these photographs from the fore...,START_ उन्होंने जमीन से इन तस्वीरों को उठाया _END,9,9
1,and after a few months went by,START_ और फिर थोड़े महीनो के बाद _END,7,8
2,really small photos just that size,START_ वाकई छोटे फोटो बस इतने बड़े _END,6,8
3,so having experimented in all sorts of remote ...,START_ तो हर तरह की दूरदराज़ जगहों पर प्रयोग क...,9,13
4,that very early on into her treatment,START_ कि इलाज के बहुत पहले _END,7,7


In [20]:
print(lines[lines['length_en'] > 20].shape)
print(lines[lines['length_hi'] > 20].shape)

(1, 4)
(334, 4)


In [21]:
lines = lines[lines['length_en'] <= 20]
lines = lines[lines['length_hi'] <= 20]
print(lines.shape)

(38469, 4)


In [22]:
print("Maximum sequence length of inputs =", max(lines['length_en']))
print("Maximum sequence length of outputs =", max(lines['length_hi']))

Maximum sequence length of inputs = 20
Maximum sequence length of outputs = 20


In [23]:
max_encoder_seq_length = max(lines['length_en'])
max_decoder_seq_length = max(lines['length_hi'])

input_words = sorted(list(en))
target_words = sorted(list(hi))
num_encoder_tokens = len(en)
num_decoder_tokens = len(hi)
num_encoder_tokens, num_decoder_tokens

(17759, 22782)

In [24]:
num_encoder_tokens += 1
num_decoder_tokens += 1

In [25]:
input_token_id = dict([(word, i + 1) for i, word in enumerate(input_words)])
target_token_id = dict([(word, i + 1) for i, word in enumerate(target_words)])

rev_input_char_id = dict((i, word) for word, i in input_token_id.items())
rev_target_char_id = dict((i, word) for word, i in target_token_id.items())

In [26]:
lines = shuffle(lines)
lines.head()

,english_sentence,hindi_sentence,length_en,length_hi
24661,“why do we have to take pops underwear back now,START_ “हमें पापा के कच्छे आज ही क्यों लौटाने ...,10,11
25990,because they have an entire economic system,START_ उनकी पूरी आर्थिक व्यवस्था जोकि _END,7,7
30422,all of these characteristics in spades,START_ इन सभी विशेशताओं के काफी समान है _END,6,9
22827,he just wanted to share a rainbow,START_ वह सिर्फ़ सबको इंद्रधनुष दिखाना चाहता थ...,7,9
16429,if i give you the letter q,START_ यदि मैं आपको एक वर्ण Q दूँ _END,7,9


# Splitting the data set

In [27]:
# Splitting data set into 80/20 for train/test data
split = int(0.8 * len(lines))

train_set = lines[: split]
test_set = lines[split :]

x_train, y_train = train_set['english_sentence'], train_set['hindi_sentence']
x_test, y_test = test_set['english_sentence'], test_set['hindi_sentence']

x_train.shape, x_test.shape

((30775,), (7694,))

In [28]:
def generate_batch(x = x_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(x), batch_size):
            encoder_input_data = np.zeros((batch_size, max_encoder_seq_length), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_decoder_seq_length), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens), dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(x[j:j + batch_size], y[j:j + batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_id[word]   # encoder input seq
                for t, word in enumerate(target_text.split()):    
                    if t < len(target_text.split()) - 1:
                        decoder_input_data[i, t] = target_token_id[word]   # decoder input seq
                    if t > 0:
                        # does not include the START_ token
                        decoder_target_data[i, t - 1, target_token_id[word]] = 1. 
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Making the Model

In [29]:
latent_dim = 256

In [30]:
encoder_inputs = Input(shape=(None,))  # Create the input tensor
encoder_embedding =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
# We discard encoder_outputs and only keep the states
encoder_states = [state_h, state_c]

In [31]:
# Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
 
# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [32]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    4546560     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    5832448     ['input_2[0][0]']                
                                                                                              

In [34]:
train_samples = len(x_train)
val_samples = len(x_test)
batch_size = 128
epochs = 100

In [35]:
model.fit(
    generate_batch(x_train, y_train, batch_size = batch_size),
    steps_per_epoch = train_samples//batch_size,
    epochs = epochs,
    validation_data = generate_batch(x_test, y_test, batch_size = batch_size),
    validation_steps = val_samples//batch_size)

Epoch 1/100
240/240 [==============================] - 63s 210ms/step - loss: 3.1603 - val_loss: 2.9141
Epoch 2/100
240/240 [==============================] - 48s 200ms/step - loss: 2.7639 - val_loss: 2.7371
Epoch 3/100
240/240 [==============================] - 49s 206ms/step - loss: 2.5896 - val_loss: 2.6566
Epoch 4/100
240/240 [==============================] - 48s 200ms/step - loss: 2.4692 - val_loss: 2.6037
Epoch 5/100
240/240 [==============================] - 49s 203ms/step - loss: 2.3631 - val_loss: 2.5573
Epoch 6/100
240/240 [==============================] - 50s 207ms/step - loss: 2.2623 - val_loss: 2.5267
Epoch 7/100
240/240 [==============================] - 49s 203ms/step - loss: 2.1634 - val_loss: 2.5033
Epoch 8/100
240/240 [==============================] - 49s 204ms/step - loss: 2.0699 - val_loss: 2.4850
Epoch 9/100
240/240 [==============================] - 49s 203ms/step - loss: 1.9790 - val_loss: 2.4705
Epoch 10/100
240/240 [==============================] - 50s 210m

In [36]:
model.save_weights('nmt_weights.h5')

In [37]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate probability over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [38]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_id['START_']

    # Sampling loop for a batch of sequences
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = rev_target_char_id[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [39]:
train_gen = generate_batch(x_train, y_train, batch_size = 1)
k=-1

In [40]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 20ms/step
Input English sentence: “why do we have to take pops underwear back now
Actual Hindi Translation:  “हमें पापा के कच्छे आज ही क्यों लौटाने हैं 
Predicted Hindi Translation:  “हमें पापा के कच्छे आज ही क्यों लौटाने हैं 


In [41]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 20ms/step
Input English sentence: because they have an entire economic system
Actual Hindi Translation:  उनकी पूरी आर्थिक व्यवस्था जोकि 
Predicted Hindi Translation:  उनकी पूरी आर्थिक व्यवस्था जोकि 


In [42]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 16ms/step
Input English sentence: all of these characteristics in spades
Actual Hindi Translation:  इन सभी विशेशताओं के काफी समान है 
Predicted Hindi Translation:  इन सभी विशेशताओं के काफी समान है 


In [43]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input English sentence: he just wanted to share a rainbow
Actual Hindi Translation:  वह सिर्फ़ सबको इंद्रधनुष दिखाना चाहता था। 
Predicted Hindi Translation:  वह सिर्फ़ सबको इंद्रधनुष दिखाना चाहता था। 


In [44]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual Hindi Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 24ms/step
Input English sentence: if i give you the letter q
Actual Hindi Translation:  यदि मैं आपको एक वर्ण Q दूँ 
Predicted Hindi Translation:  यदि मैं आपको एक वर्ण Q दूँ 
